Upload the data files into colab notebook drive before running the code.

In [5]:
import math
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

In [6]:
def data(diet_file='Personalized_Diet_Recommendations', nutrients_file='nutrients_csvfile'):
    diet_df = pd.read_csv(f"/content/{diet_file}.csv")
    nutrients_df = pd.read_csv(f"/content/{nutrients_file}.csv")

    diet_df.columns = diet_df.columns.str.strip().str.lower().str.replace(" ", "_")
    nutrients_df.columns = nutrients_df.columns.str.strip().str.lower().str.replace(" ", "_")

    diet_df.fillna({"chronic_disease": "None", "allergies": "None", "food_aversions": "None"}, inplace=True)

    label_encoders = {}

    for col in diet_df.select_dtypes(include=['object']).columns:
        if col not in ["patient_id"]:
            le = LabelEncoder()
            sorted_classes = sorted(diet_df[col].unique())
            le.fit(sorted_classes)
            diet_df[col] = le.transform(diet_df[col])
            label_encoders[col] = le

    nutrients_df.replace('t', 0, inplace=True)
    nutrients_df['protein'] = pd.to_numeric(nutrients_df['protein'], errors='coerce')
    nutrients_df['fat'] = pd.to_numeric(nutrients_df['fat'], errors='coerce')
    nutrients_df['carbs'] = pd.to_numeric(nutrients_df['carbs'], errors='coerce')
    nutrients_df['calories'] = pd.to_numeric(nutrients_df['calories'], errors='coerce')
    nutrients_df.fillna(0, inplace=True)

    return diet_df, nutrients_df, label_encoders

In [7]:
def model(diet_df, label_encoders):
    cols_to_drop = ["patient_id", "dietary_habits", "preferred_cuisine", "food_aversions", "recommended_meal_plan", "recommended_calories", "recommended_protein", "recommended_carbs", "recommended_fats"]
    features = [col for col in diet_df.columns if col not in cols_to_drop]

    X = diet_df.drop(columns=cols_to_drop)
    y = diet_df[["recommended_calories", "recommended_protein", "recommended_carbs", "recommended_fats", "recommended_meal_plan"]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.001, random_state=14)

    multi_output_regressor = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=14))
    multi_output_regressor.fit(X_train, y_train)

    with open('multi_output_regressor.pkl', 'wb') as file:
        pickle.dump(multi_output_regressor, file)

    print("Model saved to multi_output_regressor.pkl")
    return multi_output_regressor, features

In [8]:
diet_df, nutrients_df, label_encoders = data(diet_file='Personalized_Diet_Recommendations', nutrients_file='nutrients_csvfile')
multi_output_regressor, features = model(diet_df, label_encoders)

Model saved to multi_output_regressor.pkl


In [9]:
import math
import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")


def load_model(model_path="multi_output_regressor.pkl"):
    with open(model_path, "rb") as file:
        model = pickle.load(file)
    return model


def data(
    diet_file="Personalized_Diet_Recommendations.csv",
    nutrients_file="nutrients_csvfile.csv"
):
    diet_df = pd.read_csv(diet_file)
    nutrients_df = pd.read_csv(nutrients_file)

    diet_df.columns = diet_df.columns.str.strip().str.lower().str.replace(" ", "_")
    nutrients_df.columns = (
        nutrients_df.columns.str.strip().str.lower().str.replace(" ", "_")
    )

    diet_df.fillna(
        {"chronic_disease": "None", "allergies": "None", "food_aversions": "None"},
        inplace=True,
    )

    label_encoders = {}

    for col in diet_df.select_dtypes(include=["object"]).columns:
        if col not in ["patient_id"]:
            le = LabelEncoder()
            sorted_classes = sorted(diet_df[col].unique())
            le.fit(sorted_classes)
            diet_df[col] = le.transform(diet_df[col])
            label_encoders[col] = le

    nutrients_df.replace("t", 0, inplace=True)
    nutrients_df["protein"] = pd.to_numeric(nutrients_df["protein"], errors="coerce")
    nutrients_df["fat"] = pd.to_numeric(nutrients_df["fat"], errors="coerce")
    nutrients_df["carbs"] = pd.to_numeric(nutrients_df["carbs"], errors="coerce")
    nutrients_df["calories"] = pd.to_numeric(nutrients_df["calories"], errors="coerce")
    nutrients_df.fillna(0, inplace=True)

    return diet_df, nutrients_df, label_encoders


def predict_for_patient(patient_id, diet_df, model, features, label_encoders):
    patient_data = diet_df[diet_df["patient_id"] == patient_id].iloc[0].to_dict()
    if not patient_data:
        return {"error": "Patient ID not found."}

    patient_features = {col: patient_data[col] for col in features}
    new_data = pd.DataFrame([patient_features])

    for col in new_data.columns:
        if col in label_encoders:
            new_data[col] = new_data[col].apply(
                lambda x: (
                    label_encoders[col].transform([x])[0]
                    if x in list(label_encoders[col].classes_)
                    else label_encoders[col].transform([label_encoders[col].classes_[0]])[0]
                )
            )

    predicted_continuous = model.predict(new_data)

    if "recommended_meal_plan" in label_encoders:
        meal_plan = math.floor(predicted_continuous[0][4])
        predicted_categorical = label_encoders["recommended_meal_plan"].inverse_transform([meal_plan])[0]

    predictions = {
        "calories": predicted_continuous[0][0],
        "protein": predicted_continuous[0][1],
        "carbs": predicted_continuous[0][2],
        "fat": predicted_continuous[0][3],
        "recommended_meal_plan": predicted_categorical,
    }

    return predictions


def generate_meal_plans(nutrients_df, recommended_values):
    meal_plans = {}

    meal_targets = {
        "Breakfast": {
            "calories": recommended_values["calories"] * 0.3,
            "protein": recommended_values["protein"] * 0.3,
            "carbs": recommended_values["carbs"] * 0.3,
            "fat": recommended_values["fat"] * 0.3,
        },
        "Lunch": {
            "calories": recommended_values["calories"] * 0.4,
            "protein": recommended_values["protein"] * 0.4,
            "carbs": recommended_values["carbs"] * 0.4,
            "fat": recommended_values["fat"] * 0.4,
        },
        "Dinner": {
            "calories": recommended_values["calories"] * 0.3,
            "protein": recommended_values["protein"] * 0.3,
            "carbs": recommended_values["carbs"] * 0.3,
            "fat": recommended_values["fat"] * 0.3,
        },
    }

    meal_categories = {
        "Breakfast": [
            "Dairy products",
            "Breads, cereals, grains",
            "Fruits A-F",
            "Fruits G-P",
            "Fruits R-Z",
            "Seeds and Nuts",
            "Jams",
            "Jellies",
        ],
        "Lunch": [
            "Soups",
            "Meat",
            "Poultry",
            "Vegetables A-E",
            "Vegetables F-P",
            "Vegetables R-Z",
            "Breads, cereals, fastfood, grains",
        ],
        "Dinner": [
            "Soups",
            "Fish",
            "Seafood",
            "Vegetables A-E",
            "Vegetables F-P",
            "Vegetables R-Z",
            "Breads, cereals, fastfood, grains",
            "Desserts",
        ],
    }

    for meal, targets in meal_targets.items():
        meal_foods = nutrients_df[nutrients_df["category"].isin(meal_categories[meal])]

        meal_plan = []
        total_calories = 0
        total_protein = 0
        total_carbs = 0
        total_fat = 0

        meal_foods = meal_foods.sort_values(by="calories", ascending=False)

        for _, row in meal_foods.iterrows():
            if (
                total_calories + row["calories"] <= targets["calories"]
                and total_protein + row["protein"] <= targets["protein"]
                and total_carbs + row["carbs"] <= targets["carbs"]
                and total_fat + row["fat"] <= targets["fat"]
            ):
                meal_plan.append(row)

                total_calories += row["calories"]
                total_protein += row["protein"]
                total_carbs += row["carbs"]
                total_fat += row["fat"]

        meal_plans[meal] = {
            "foods": meal_plan,
            "totals": {
                "calories": total_calories,
                "protein": total_protein,
                "carbs": total_carbs,
                "fat": total_fat,
            },
        }

    return meal_plans


def get_recommendations(
    patient_id,
    model_path="multi_output_regressor.pkl",
    diet_file="Personalized_Diet_Recommendations.csv",
    nutrients_file="nutrients_csvfile.csv",
):
    model = load_model(model_path)

    diet_df, nutrients_df, label_encoders = data(diet_file, nutrients_file)

    cols_to_drop = [
        "patient_id",
        "dietary_habits",
        "preferred_cuisine",
        "food_aversions",
        "recommended_meal_plan",
        "recommended_calories",
        "recommended_protein",
        "recommended_carbs",
        "recommended_fats",
    ]

    features = [col for col in diet_df.columns if col not in cols_to_drop]

    predictions = predict_for_patient(patient_id, diet_df, model, features, label_encoders)

    meal_plans = generate_meal_plans(nutrients_df, predictions)

    return {"predictions": predictions, "meal_plans": meal_plans}

In [10]:
if __name__ == "__main__":
    patient_id = "P01234"
    model_path = "/content/multi_output_regressor.pkl"
    diet_file = "/content/Personalized_Diet_Recommendations.csv"
    nutrients_file = "/content/nutrients_csvfile.csv"
    recommendations = get_recommendations(patient_id, model_path, diet_file, nutrients_file)

In [11]:
print("Predictions:")
print("------------")
print(f"Calories: {recommendations['predictions']['calories']}")
print(f"Protein: {recommendations['predictions']['protein']}g")
print(f"Carbs: {recommendations['predictions']['carbs']}g")
print(f"Fat: {recommendations['predictions']['fat']}g")
print(f"Recommended Meal Plan: {recommendations['predictions']['recommended_meal_plan']}")
print("\n")

print("Meal Plans:")
print("-----------")
for meal, plan in recommendations["meal_plans"].items():
    print(f"\n{meal}:")
    print("Foods:")
    for food in plan["foods"]:
        print(f"- {food['food']} ({food['measure']}): {food['calories']} kcal, {food['protein']}g protein, {food['carbs']}g carbs, {food['fat']}g fat")
    print("\nTotals:")
    print(f"Calories: {plan['totals']['calories']}")
    print(f"Protein: {plan['totals']['protein']}g")
    print(f"Carbs: {plan['totals']['carbs']}g")
    print(f"Fat: {plan['totals']['fat']}g")

Predictions:
------------
Calories: 3109.29
Protein: 98.79g
Carbs: 335.27g
Fat: 132.31g
Recommended Meal Plan: Balanced Diet


Meal Plans:
-----------

Breakfast:
Foods:
- (1/2 cup ice cream) (2 cups): 690.0 kcal, 24g protein, 70.0g carbs, 24.0g fat
- Cream or half-and-half (1/2 cup): 170.0 kcal, 4g protein, 5.0g carbs, 15.0g fat
- Grapes (1 cup): 70.0 kcal, 1g protein, 16.0g carbs, 0.0g fat

Totals:
Calories: 930.0
Protein: 29g
Carbs: 91.0g
Fat: 39.0g

Lunch:
Foods:
- Potatoes, pan-tried (3/4 cup): 268.0 kcal, 4g protein, 33.0g carbs, 14.0g fat
- Lima, dry, cooked (1 cup): 260.0 kcal, 16g protein, 48.0g carbs, 0.0g fat
- Peppers with beef and crumbs (1 med.): 255.0 kcal, 19g protein, 24.0g carbs, 9.0g fat
- French-fried (10 pieces): 155.0 kcal, -1g protein, 20.0g carbs, 7.0g fat
- Turnips, steamed (1 cup): 40.0 kcal, 1g protein, 9.0g carbs, 0.0g fat
- Parsley (2 T.): 2.0 kcal, 0g protein, 0.0g carbs, 0.0g fat

Totals:
Calories: 980.0
Protein: 39g
Carbs: 134.0g
Fat: 30.0g

Dinner:
Food

#

#